# Bayesian Model

This notebook illustrate how to use a Bayesian Normal density model with the [beer framework](https://github.com/beer-asr/beer). The Normal distribution is a fairly basic models but it is used extenslively in other model as a basic building block.

In [1]:
# Add "beer" to the PYTHONPATH
import sys
sys.path.insert(0, '../')

import beer
import numpy as np
import torch

# For plotting.
from bokeh.io import show, output_notebook
from bokeh.plotting import figure, gridplot
output_notebook()

# Convenience functions for plotting.
import plotting

%load_ext autoreload
%autoreload 2

Loading BokehJS ...

## Data

Generate some normally distributed data:

In [2]:
mean = np.array([-1.5, 4]) 
cov = np.array([
    [2, 1],
    [1, .75]
])
data = np.random.multivariate_normal(mean, cov, size=100)

fig = figure(
    title='Data',
    width=400,
    height=400,
    x_range=(mean[0] - 5, mean[0] + 5),
    y_range=(mean[1] - 5, mean[1] + 5)
)
fig.circle(data[:, 0], data[:, 1])
plotting.plot_normal(fig, mean, cov, alpha=.5, color='black')

show(fig)

## Model Creation

We create two types of Normal distribution: one diagonal covariance matrix and another one with full covariance matrix.

In [3]:
normal_diag = beer.NormalDiagonalCovariance(
    prior=beer.NormalGammaPrior(torch.zeros(2), torch.ones(2), 1.),
    posterior=beer.NormalGammaPrior(torch.zeros(2), torch.ones(2), 1.)
)
normal_full = beer.NormalFullCovariance(
    prior=beer.NormalWishartPrior(torch.zeros(2), torch.eye(2), 1.),
    posterior=beer.NormalWishartPrior(torch.zeros(2), torch.eye(2), 1.)
)

## Variational Bayes Training 

In [5]:
epochs = 1
lrate = 1
X = torch.from_numpy(data).float()
loss_fn = beer.StochasticVariationalBayesLoss(len(X))
params = normal_diag.parameters + normal_full.parameters
optimizer = beer.BayesianModelOptimizer(params, lrate)
    
for epoch in range(epochs):
    optimizer.zero_grad()
    loss_ndiag = loss_fn(normal_diag, X)
    loss_nfull = loss_fn(normal_full, X)
    loss_ndiag.backward_natural_grad()
    loss_nfull.backward_natural_grad()
    optimizer.step()

In [6]:
fig = figure(
    width=400,
    height=400,
    x_range=(mean[0] - 5, mean[0] + 5),
    y_range=(mean[1] - 5, mean[1] + 5)
)
fig.circle(data[:, 0], data[:, 1])
plotting.plot_normal(fig, normal_diag.mean.numpy(), normal_diag.cov.numpy(), alpha=.5, color='red')
plotting.plot_normal(fig, normal_full.mean.numpy(), normal_full.cov.numpy(), alpha=.5, color='blue')

show(fig)

/Users/lucasondel/anaconda3/envs/pytorch_src/lib/python3.6/site-packages/bokeh/core/json_encoder.py:80: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  elif np.issubdtype(type(obj), np.float):
